In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-sentiment/train_2kmZucJ.csv
/kaggle/input/test-sentiment/test_oJQbWVk.csv
/kaggle/input/sample-sub-sentiment/sample_submission_LnhVWA4.csv


In [47]:
train=pd.read_csv('/kaggle/input/training-sentiment/train_2kmZucJ.csv')
test=pd.read_csv('/kaggle/input/test-sentiment/test_oJQbWVk.csv')
samp_sub=pd.read_csv('/kaggle/input/sample-sub-sentiment/sample_submission_LnhVWA4.csv')

In [48]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [49]:
train['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [50]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [51]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from PIL import Image

In [52]:
train.drop('id',axis=1,inplace=True)

In [53]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import fasttext
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

In [54]:
df=train.copy()

In [55]:
def textblob_score(sentence):
    return TextBlob(sentence).sentiment.polarity

# Convert textblob sentiment score
df['textblob_score'] = df['tweet'].apply(textblob_score)
# Convert float score to category based on binning
df['textblob_pred'] = pd.cut(df['textblob_score'], bins=2, labels=[1,0])
df = df.drop('textblob_score', axis=1)
df.head()

,label,tweet,textblob_pred
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,0
1,0,Finally a transparant silicon case ^^ Thanks t...,0
2,0,We love this! Would you go? #talk #makememorie...,0
3,0,I'm wired I know I'm George I was made that wa...,0
4,1,What amazing service! Apple won't even talk to...,1


In [56]:
def print_accuracy(df, pred_column):
    "Print f1 score and accuracy after making predictions"
    f1_macro = f1_score(df['label'], df[pred_column], average='macro')
    acc = accuracy_score(df['label'], df[pred_column])*100
    return f1_macro, acc

In [57]:
acc = print_accuracy(df, 'textblob_pred')
print("Macro F1-score: {}\nAccuracy: {}".format(acc[0], acc[1]))

Macro F1-score: 0.6191381893938928
Accuracy: 65.31565656565657


In [58]:
df_test=test.copy()

In [59]:
df_test['textblob_score'] = df_test['tweet'].apply(textblob_score)
# Convert float score to category based on binning
df_test['textblob_pred'] = pd.cut(df_test['textblob_score'], bins=2, labels=[1,0])
df_test = df_test.drop('textblob_score', axis=1)
df_test.head()

,id,tweet,textblob_pred
0,7921,I hate the new #iphone upgrade. Won't let me d...,1
1,7922,currently shitting my fucking pants. #apple #i...,1
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",1
3,7924,My ipod is officially dead. I lost all my pict...,1
4,7925,Been fighting iTunes all night! I only want th...,1


In [60]:
df_test['textblob_pred'].value_counts()

0    1049
1     904
Name: textblob_pred, dtype: int64

In [61]:
df1=train.copy()

In [62]:
vader = SentimentIntensityAnalyzer()
def score_vader(sentence, vader):
    return vader.polarity_scores(sentence)['compound']

df1['vader_score'] = df1['tweet'].apply(lambda x: score_vader(x, vader))
# Convert float score to category based on binning
df1['vader_pred'] = pd.cut(df1['vader_score'], bins=2, labels=[1,0])
df1.head()

,label,tweet,vader_score,vader_pred
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,0.0000,1
1,0,Finally a transparant silicon case ^^ Thanks t...,0.7096,0
2,0,We love this! Would you go? #talk #makememorie...,0.6696,0
3,0,I'm wired I know I'm George I was made that wa...,0.2263,0
4,1,What amazing service! Apple won't even talk to...,0.5080,0


In [63]:
acc = print_accuracy(df1, 'vader_pred')
print("Macro F1-score: {}\nAccuracy: {}".format(acc[0], acc[1]))

Macro F1-score: 0.49766856035939533
Accuracy: 50.83333333333333


In [64]:
df2=train.copy()

In [65]:
df_test['label']=df_test['textblob_pred']
df_test.drop(['tweet','textblob_pred'],axis=1,inplace=True)

In [66]:
df_test.to_csv('textblob_sub1.csv',index=False)

In [67]:
df_test1=test.copy()

In [69]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pipeline = Pipeline(
    [
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', LogisticRegression(solver='liblinear', multi_class='auto')),
    ]
)


learner = pipeline.fit(df2['tweet'],df2['label'])

df_test1['pred'] = learner.predict(df_test1['tweet'])
df_test1.head()

,id,tweet,pred
0,7921,I hate the new #iphone upgrade. Won't let me d...,1
1,7922,currently shitting my fucking pants. #apple #i...,0
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",1
3,7924,My ipod is officially dead. I lost all my pict...,1
4,7925,Been fighting iTunes all night! I only want th...,1


In [ ]:
df_test1.to_csv('logistic_r_sub2.csv',index=False)